In [2]:
import yfinance as yf
import pandas as pandas
import numpy as numpy
import matplotlib.pyplot as pyplot
import cvxpy as cvxpy


In [10]:
# 50 largest companies in Sweden
tickers = [
    'INVE-B.ST', 'ATCO-B.ST', 'VOLV-B.ST', 'ASSA-B.ST', 'EQT.ST',
    'SEB-A.ST', 'ERIC-B.ST', 'HEXA-B.ST', 'SAND.ST', 'HM-B.ST',
    'SWED-A.ST', 'EPI-A.ST', 'SHB-A.ST', 'ESSITY-B.ST', 'ALFA.ST',
    'EVO.ST', 'LATO-B.ST', 'INDU-A.ST', 'LIFCO-B.ST',
    'LUND-B.ST', 'SAAB-B.ST', 'TELIA.ST', 'AKEL-D.ST',
    'SOBI.ST', 'INDT.ST', 'SCA-B.ST', 'SKF-B.ST', 'SKA-B.ST',
    'BEIJ-B.ST', 'ALIV-SDB.ST', 'SAVE.ST', 'HOLM-B.ST', 'AAK.ST',
    'SWEC-B.ST', 'SECT-B.ST', 'SSAB-B.ST', 'AXFO.ST', 'GETI-B.ST',
    'SAGA-B.ST', 'VOLCAR-B.ST', 'NDA-SE.ST', 'KINV-B.ST', 'BOL.ST',
    'FABG.ST', 'BALD-B.ST', 'CAST.ST', 'PEAB-B.ST', 'NCC-B.ST'
]

# Reference index : OMX Stockholm 30
benchmark_ticker = '^OMX'  

# Analysis period, can easily be changed
start_date = "2010-01-01"
end_date = "2024-11-30"

# Download historical closing prices
data = yf.download(tickers + [benchmark_ticker], start=start_date, end=end_date)['Close']

# Calculation of historical returns
returns = data[tickers].pct_change().dropna()
benchmark_returns = data[benchmark_ticker].pct_change().dropna()

# Defining in- and out-of-sample
split_date = "2019-01-01"
returns_in = returns.loc[:split_date]
returns_out = returns.loc[split_date:]

[*********************100%***********************]  49 of 49 completed
C:\Users\steha46\AppData\Local\Temp\ipykernel_11096\820961771.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  benchmark_returns = data[benchmark_ticker].pct_change().dropna()


In [12]:
print(returns)

Ticker      INVE-B.ST  ATCO-B.ST  VOLV-B.ST  ASSA-B.ST    EQT.ST  SEB-A.ST  \
Date                                                                         
2021-11-01   0.015412   0.003660   0.000900   0.005964 -0.010164  0.011546   
2021-11-02   0.006220   0.000000   0.007996   0.011858  0.028795  0.010678   
2021-11-03   0.016568   0.005792   0.011651   0.009766  0.031677  0.016393   
2021-11-04   0.015811   0.003839   0.002450   0.001547  0.032597 -0.008244   
2021-11-05   0.006496  -0.014659   0.006111  -0.014677  0.018737  0.014095   
...               ...        ...        ...        ...       ...       ...   
2024-11-25  -0.002681  -0.007507  -0.006630  -0.001805  0.032888 -0.001989   
2024-11-26  -0.003360   0.001973  -0.007787  -0.012059 -0.011980 -0.010628   
2024-11-27   0.001854  -0.002297   0.006726  -0.001526  0.007339  0.001007   
2024-11-28   0.001514   0.000987   0.011136   0.014364  0.000317  0.014084   
2024-11-29   0.005376   0.012816  -0.002937   0.009039  0.052548